In [ ]:
with engine.begin() as conn: 
    result = conn.execute(text(f'''
                               SELECT * FROM product_analysis; 
                                '''))
    data = result.all()

### Let's create a dataframe out of that
product_analysis = pd.DataFrame(data) 
product_analysis

In [ ]:
# Aggregate data by product type and color group to summarize demand
product_data = product_analysis.groupby(['product_type_no', 'colour_group_code', 'garment_group_no']).agg({
    'units_sold': 'sum'  # Aggregated demand
}).reset_index()

In [ ]:
from sklearn.preprocessing import StandardScaler

# Select features for scaling
features = product_data[['units_sold', 'product_type_no', 'colour_group_code', 'garment_group_no']]
scaler = StandardScaler()
scaled_data = scaler.fit_transform(features)

In [ ]:
import hdbscan

# Initialize HDBSCAN with chosen parameters
hdbscan_clusterer = hdbscan.HDBSCAN(min_samples=5, min_cluster_size=10)
product_clusters = hdbscan_clusterer.fit_predict(scaled_data)

# Add cluster labels back to the data
product_data['cluster'] = product_clusters

In [ ]:
import numpy as np

# Display cluster distribution
unique, counts = np.unique(product_clusters, return_counts=True)
cluster_distribution = dict(zip(unique, counts))
print("Cluster distribution:", cluster_distribution)


In [ ]:
# Summarize average demand and characteristics by cluster
cluster_summary = product_data.groupby('cluster').mean()
print(cluster_summary)